In [ ]:
import json
import chromadb
from sentence_transformers import SentenceTransformer
from transformers import pipeline

In [ ]:
# Connect to the persistent ChromaDB database
client = chromadb.PersistentClient(path="pdf-to-text-chroma-search/db")
collection = client.get_collection(name="my_collection")

# Load the embedding model (using SentenceTransformer)
model = SentenceTransformer("louisbrulenaudet/lemone-gte-embed-max", trust_remote_code=True)

# Load the question-answering pipeline
qa_pipeline = pipeline("question-answering", model="cmarkea/distilcamembert-base-qa", tokenizer="cmarkea/distilcamembert-base-qa")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/31.3k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
json_file_path = "pdf-to-text-chroma-search/updated_questions.json"  # adjust the path as needed

with open(json_file_path, "r", encoding="utf-8") as f:
    data = json.load(f)

In [ ]:
for item in data["questions"]:
    question = item["question"]
    real_answer = item["answer"]

    # 1. Query the vector database with the question
    query_vector = model.encode(question)
    results = collection.query(query_embeddings=query_vector, n_results=4, include=["documents"])

    # 2. Concatenate the retrieved documents to form the context
    context = " ".join([doc for docs in results["documents"] for doc in docs])

    # 3. Use the QA pipeline to get an answer using the question and the context
    result = qa_pipeline(question=question, context=context)

    # 4. Print the question, the LLM answer with its score, and the expert answer from JSON
    print("Question:", question)
    print("LLM Answer:", result['answer'])
    print("Score:", result['score'])
    print("Real Answer:", real_answer)
    print("-" * 80)

Question: Quelles sont les entreprises bénéficiaires du régime suspensif ?
LLM Answer:  d'ouvrir droit à 
déduction et au remboursement prévus
Score: 0.022826634347438812
Real Answer: Conformément aux dispositions de l'article 94 du Code Général des Impôts (CGI), le bénéfice du régime suspensif est réservé uniquement : 
• aux entreprises exportatrices de produits qui, sur leur demande et dans la limite du chiffre d'affaires réalisé au cours de l'année écoulée au titre de leurs opérations d'exportation, peuvent être autorisées à recevoir en suspension de la TVA à l'intérieur, les marchandises, les matières premières, les emballages irrécupérables ainsi que les services nécessaires auxdites opérations et susceptibles d'ouvrir droit à déduction et au remboursement prévus aux articles 101 à 104 du CGI ; 
• aux entreprises exportatrices de services qui, sur leur demande et dans la limite du chiffre d'affaires réalisé au cours de l'année écoulée au titre de leurs opérations d'exportation, pe